In [952]:
import pandas as pd

In [953]:
census_df = pd.read_csv("./acs2017_census_tract_data.csv", 
                        usecols=['County', 'State', 'TotalPop', 
                                 'Poverty', 'IncomePerCap', 'TractId'])
covid_df = pd.read_csv("./COVID_county_data.csv", usecols=['date','county','state','cases','deaths'])

census_renamer = {
    "County": "Name",
    "TotalPop": "Population",
    "IncomePerCap": "PerCapitaIncome",
    "TractID": "ID",
}

covid_renamer = {
    "date": "Month",
    "cases": "Cases",
    "deaths": "Deaths",
    "county": "Name",
    "state": "State",
}

In [954]:
census_df = census_df.rename(columns=census_renamer)
covid_df = covid_df.rename(columns=covid_renamer)

In [955]:
print(census_df.columns)
print(covid_df.columns)

Index(['TractId', 'State', 'Name', 'Population', 'PerCapitaIncome', 'Poverty'], dtype='object')
Index(['Month', 'Name', 'State', 'Cases', 'Deaths'], dtype='object')


In [956]:
census_df.head(2)

,TractId,State,Name,Population,PerCapitaIncome,Poverty
0,1001020100,Alabama,Autauga County,1845,33018.0,10.7
1,1001020200,Alabama,Autauga County,2172,18996.0,22.4


In [957]:
census_list = census_df[["Name", "State"]].copy()
county_info = census_df.groupby(['Name', 'State'])['Population'].sum().reset_index().sort_values(by=['State', 'Name'])
county_info = county_info.reset_index().drop(columns=['index'])

In [958]:
county_info

,Name,State,Population
0,Autauga County,Alabama,55036
1,Baldwin County,Alabama,203360
2,Barbour County,Alabama,26201
3,Bibb County,Alabama,22580
4,Blount County,Alabama,57667
...,...,...,...
3215,Sweetwater County,Wyoming,44527
3216,Teton County,Wyoming,22923
3217,Uinta County,Wyoming,20758
3218,Washakie County,Wyoming,8253


In [959]:
poverty_group = census_df.groupby(['Name', 'State'])['Poverty'].sum().reset_index().sort_values(by=['State', 'Name'])
county_entry_count = census_df.groupby(['Name', 'State']).size().reset_index(name='EntryCount').sort_values(by=['State', 'Name'])
poverty_group['Poverty'] /= county_entry_count['EntryCount']
poverty_group = poverty_group.reset_index().drop(columns=['index'])

In [960]:
percap_group = census_df.groupby(['Name', 'State'])['PerCapitaIncome'].sum().reset_index().sort_values(by=['State', 'Name'])
per_cap_entry = census_df.groupby(['Name', 'State']).size().reset_index(name='PerCapEntry').sort_values(by=['State', 'Name'])
percap_group['PerCapitaIncome'] /= per_cap_entry['PerCapEntry']
percap_group = percap_group.reset_index().drop(columns=['index'])

In [961]:
county_info['Poverty'] = poverty_group['Poverty']
county_info['PerCapitaIncome'] = percap_group['PerCapitaIncome']

In [962]:
county_info

,Name,State,Population,Poverty,PerCapitaIncome
0,Autauga County,Alabama,55036,14.558333,26588.166667
1,Baldwin County,Alabama,203360,12.471875,28220.375000
2,Barbour County,Alabama,26201,27.755556,17891.666667
3,Bibb County,Alabama,22580,13.925000,21799.000000
4,Blount County,Alabama,57667,16.422222,21598.444444
...,...,...,...,...,...
3215,Sweetwater County,Wyoming,44527,11.691667,32898.083333
3216,Teton County,Wyoming,22923,7.550000,47706.250000
3217,Uinta County,Wyoming,20758,14.733333,27086.000000
3218,Washakie County,Wyoming,8253,13.066667,26828.333333


In [963]:
county_info[(county_info['Name'] == 'Loudoun County') & (county_info['State'] == 'Virginia')]

,Name,State,Population,Poverty,PerCapitaIncome
2968,Loudoun County,Virginia,374558,3.824615,49615.769231


In [964]:
county_info[(county_info['Name'] == 'Washington County') & (county_info['State'] == 'Oregon')]

,Name,State,Population,Poverty,PerCapitaIncome
2241,Washington County,Oregon,572071,10.446154,34970.817308


In [965]:
county_info[(county_info['Name'] == 'Harlan County') & (county_info['State'] == 'Kentucky')]

,Name,State,Population,Poverty,PerCapitaIncome
1040,Harlan County,Kentucky,27548,33.318182,16010.363636


In [966]:
county_info[(county_info['Name'] == 'Malheur County') & (county_info['State'] == 'Oregon')]

,Name,State,Population,Poverty,PerCapitaIncome
2230,Malheur County,Oregon,30421,21.3625,15720.625


In [967]:
covid_df['Month'] = pd.to_datetime(covid_df['Month'])
covid_df['Month'] = covid_df['Month'].dt.to_period('M')

In [968]:
monthly_cases = covid_df.groupby(['Month', 'Name', 'State'])['Cases'].sum().reset_index()
monthly_cases = monthly_cases.sort_values(by=['State', 'Name']).reset_index().drop(columns='index')

In [969]:
monthly_cases['Month'] = monthly_cases['Month'].astype(str)

In [970]:
monthly_deaths = covid_df.groupby(['Month', 'Name', 'State'])['Deaths'].sum().reset_index()
monthly_deaths = monthly_deaths.sort_values(by=['State', 'Name']).reset_index().drop(columns='index')

In [971]:
monthly_deaths['Month'] = monthly_deaths['Month'].astype(str)

In [972]:
Covid_monthly = monthly_cases.copy()
Covid_monthly['Deaths'] = monthly_deaths['Deaths']

In [973]:
Covid_monthly[(Covid_monthly['Name'] == 'Malheur') & (Covid_monthly['State'] == 'Oregon') & (Covid_monthly['Month'] == '2020-08')]

,Month,Name,State,Cases,Deaths
25897,2020-08,Malheur,Oregon,28163,459.0


In [974]:
Covid_monthly[(Covid_monthly['Name'] == 'Malheur') & (Covid_monthly['State'] == 'Oregon') & (Covid_monthly['Month'] == '2021-01')]

,Month,Name,State,Cases,Deaths
25902,2021-01,Malheur,Oregon,96297,1627.0


In [975]:
Covid_monthly[(Covid_monthly['Name'] == 'Malheur') & (Covid_monthly['State'] == 'Oregon') & (Covid_monthly['Month'] == '2021-02')]

,Month,Name,State,Cases,Deaths
25903,2021-02,Malheur,Oregon,65951,1137.0


In [976]:
cases_group = Covid_monthly.groupby(['State','Name'])['Cases'].sum().reset_index()
deaths_group = Covid_monthly.groupby(['State','Name'])['Deaths'].sum().reset_index()

In [977]:
covid_summary = county_info.copy()
covid_summary['Name'] = covid_summary['Name'].str.slice(0,-7)

In [978]:
cases_group

,State,Name,Cases
0,Alabama,Autauga,645935
1,Alabama,Baldwin,2003567
2,Alabama,Barbour,268771
3,Alabama,Bibb,261043
4,Alabama,Blount,630106
...,...,...,...
3269,Wyoming,Teton,305376
3270,Wyoming,Uinta,200783
3271,Wyoming,Unknown,37
3272,Wyoming,Washakie,84354


In [979]:
deaths_group

,State,Name,Deaths
0,Alabama,Autauga,9042.0
1,Alabama,Baldwin,23041.0
2,Alabama,Barbour,4077.0
3,Alabama,Bibb,5272.0
4,Alabama,Blount,8669.0
...,...,...,...
3269,Wyoming,Teton,617.0
3270,Wyoming,Uinta,1037.0
3271,Wyoming,Unknown,0.0
3272,Wyoming,Washakie,2622.0


In [980]:
cases_group = cases_group.merge(covid_summary, on=['State', 'Name'], how='inner')
staging_group = deaths_group.merge(cases_group, on=['State', 'Name'], how='inner')

In [981]:
staging_group

,State,Name,Deaths,Cases,Population,Poverty,PerCapitaIncome
0,Alabama,Autauga,9042.0,645935,55036,14.558333,26588.166667
1,Alabama,Baldwin,23041.0,2003567,203360,12.471875,28220.375000
2,Alabama,Barbour,4077.0,268771,26201,27.755556,17891.666667
3,Alabama,Bibb,5272.0,261043,22580,13.925000,21799.000000
4,Alabama,Blount,8669.0,630106,57667,16.422222,21598.444444
...,...,...,...,...,...,...,...
3061,Wyoming,Sweetwater,2239.0,323730,44527,11.691667,32898.083333
3062,Wyoming,Teton,617.0,305376,22923,7.550000,47706.250000
3063,Wyoming,Uinta,1037.0,200783,20758,14.733333,27086.000000
3064,Wyoming,Washakie,2622.0,84354,8253,13.066667,26828.333333


In [982]:
covid_summary = pd.DataFrame()
covid_summary[['State', 'Name', 'Population', 'Poverty', 'PerCapitaIncome', 'TotalCases', 'TotalDeaths']] = (
    staging_group[['State', 'Name', 'Population', 'Poverty', 'PerCapitaIncome', 'Cases', 'Deaths']])

In [983]:
covid_summary['TotalCasesPer100K'] = covid_summary['TotalCases'] / 100000
covid_summary['TotalDeathsPer100K'] = covid_summary['TotalDeaths'] / 100000

In [984]:
covid_summary

,State,Name,Population,Poverty,PerCapitaIncome,TotalCases,TotalDeaths,TotalCasesPer100K,TotalDeathsPer100K
0,Alabama,Autauga,55036,14.558333,26588.166667,645935,9042.0,6.45935,0.09042
1,Alabama,Baldwin,203360,12.471875,28220.375000,2003567,23041.0,20.03567,0.23041
2,Alabama,Barbour,26201,27.755556,17891.666667,268771,4077.0,2.68771,0.04077
3,Alabama,Bibb,22580,13.925000,21799.000000,261043,5272.0,2.61043,0.05272
4,Alabama,Blount,57667,16.422222,21598.444444,630106,8669.0,6.30106,0.08669
...,...,...,...,...,...,...,...,...,...
3061,Wyoming,Sweetwater,44527,11.691667,32898.083333,323730,2239.0,3.23730,0.02239
3062,Wyoming,Teton,22923,7.550000,47706.250000,305376,617.0,3.05376,0.00617
3063,Wyoming,Uinta,20758,14.733333,27086.000000,200783,1037.0,2.00783,0.01037
3064,Wyoming,Washakie,8253,13.066667,26828.333333,84354,2622.0,0.84354,0.02622


In [ ]:
covid_summary[(covid_summary['Name'] == 'Malheur') & (covid_summary['State'] == 'Oregon')]
